In [1]:
import pandas as pd


# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import the emp_aval table

# Columns Names: Emp_Name, Work_From, Work_To, Working_Flag



In [3]:
# Delete later: Creating a sample emp_avl table

import pandas as pd
import numpy as np

# Define the employee names and time range
emp_names = ['Andrew', 'Emily', 'James']
time_range = pd.date_range(start='08:00', end='11:00', freq='30T')

# Create a list to hold the data
data = []

# Create combinations of Emp_Name, From_Time, and To_Time
for emp in emp_names:
    for i in range(len(time_range) - 1):
        from_time = time_range[i].time()
        to_time = time_range[i + 1].time()
        working_flag = np.random.choice([0, 1], p=[0.3, 0.7])  # 70% chance to be 1
        data.append([emp, from_time, to_time, working_flag])

# Create the DataFrame
emp_aval = pd.DataFrame(data, columns=['Emp_Name', 'Work_From', 'Work_To', 'Working_Flag'])

# Display the DataFrame
display(emp_aval.head())


,Emp_Name,Work_From,Work_To,Working_Flag
0,Andrew,08:00:00,08:30:00,1
1,Andrew,08:30:00,09:00:00,1
2,Andrew,09:00:00,09:30:00,1
3,Andrew,09:30:00,10:00:00,0
4,Andrew,10:00:00,10:30:00,1


In [4]:
# Import emp_count_req Default Table
emp_count_req= pd.read_excel(r"00_Input\02_Emp_Count_Requirement.xlsx")
emp_count_req.head()

# Convert 'From_Time' and 'To_Time' to datetime objects
emp_count_req['From_Time'] = pd.to_datetime(emp_count_req['From_Time'], format='%H:%M:%S').dt.time
emp_count_req['To_Time'] = pd.to_datetime(emp_count_req['To_Time'], format='%H:%M:%S').dt.time

emp_count_req.head()

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed,Greeter_Up_Needed,Greeter_Down_Needed,Min_Total_Emp_Needed
0,07:30:00,08:00:00,3,3,0,0,6
1,08:00:00,08:30:00,3,3,0,0,6
2,08:30:00,09:00:00,3,3,1,1,8
3,09:00:00,09:30:00,3,3,1,1,8
4,09:30:00,10:00:00,3,3,1,1,8


In [5]:
# Create "total available employee" column

# Group By the From and To time of "Work Status Per Time" table and count the number of available employees
grouped_avl_by_time= emp_aval.groupby(['Work_From', 'Work_To']).agg({'Working_Flag': 'sum'}).reset_index()
grouped_avl_by_time.columns= ['Work_From', 'Work_To', 'Total_Avl_Emp']
display(grouped_avl_by_time.head())

# Join it with the emp_count_req table
emp_demand_check= pd.merge(emp_count_req, grouped_avl_by_time, how='left', left_on=['From_Time','To_Time'], right_on=['Work_From','Work_To'])
columns_needed= ['From_Time', 'To_Time', 'Reg_Up_Needed', 'Reg_Down_Needed', 'Greeter_Up_Needed', 'Greeter_Down_Needed', 'Min_Total_Emp_Needed', 'Total_Avl_Emp']
emp_demand_check= emp_demand_check[columns_needed]
emp_demand_check['Total_Avl_Emp'].fillna(0, inplace=True) # If no emp are available on a selected shift, make the availability zero. 

# Create an availability check flag and alert 
def alert_employee_shortage(emp_demand_check: pd.DataFrame):
    shortage= emp_demand_check[emp_demand_check['Availability_Check_Flag'] == False]
    if(len(shortage)==0):
        print("No shortage of employees for the whole day")
    else:
        print("ALERT: Employees are on shortage for the following time slots")
        print(shortage[['From_Time', 'To_Time', 'Min_Total_Emp_Needed', 'Total_Avl_Emp']])
    return

emp_demand_check['Availability_Check_Flag']= emp_demand_check['Min_Total_Emp_Needed']<= emp_demand_check['Total_Avl_Emp']
alert_employee_shortage(emp_demand_check)
display(emp_demand_check.sample(5))


,Work_From,Work_To,Total_Avl_Emp
0,08:00:00,08:30:00,1
1,08:30:00,09:00:00,2
2,09:00:00,09:30:00,2
3,09:30:00,10:00:00,1
4,10:00:00,10:30:00,2


ALERT: Employees are on shortage for the following time slots
   From_Time   To_Time  Min_Total_Emp_Needed  Total_Avl_Emp
0   07:30:00  08:00:00                     6            0.0
1   08:00:00  08:30:00                     6            1.0
2   08:30:00  09:00:00                     8            2.0
3   09:00:00  09:30:00                     8            2.0
4   09:30:00  10:00:00                     8            1.0
5   10:00:00  10:30:00                     8            2.0
6   10:30:00  11:00:00                     8            1.0
7   11:00:00  11:30:00                     8            0.0
8   11:30:00  12:00:00                     8            0.0
9   12:00:00  12:30:00                    10            0.0
10  12:30:00  13:00:00                    10            0.0
11  13:00:00  13:30:00                    10            0.0
12  13:30:00  14:00:00                    10            0.0
13  14:00:00  14:30:00                    10            0.0
14  14:30:00  15:00:00                

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed,Greeter_Up_Needed,Greeter_Down_Needed,Min_Total_Emp_Needed,Total_Avl_Emp,Availability_Check_Flag
14,14:30:00,15:00:00,5,3,1,1,10,0.0,False
15,15:00:00,15:30:00,5,3,1,1,10,0.0,False
20,17:30:00,18:00:00,2,3,1,1,7,0.0,False
11,13:00:00,13:30:00,5,3,1,1,10,0.0,False
0,07:30:00,08:00:00,3,3,0,0,6,0.0,False


In [11]:
emp_demand_check.to_excel("Intermediate_Results/Sample_Shift_Count_Rquirement_table.xlsx", index= False)

In [10]:
emp_demand_check.iloc[0,0]

datetime.time(7, 30)